In [ ]:
import os
import re

import pandas
import requests

In [ ]:
data_path = os.path.join('.', 'data_as_provided')
output_path = os.path.join('.', 'data_processed')
arc_scenarios_file = os.path.join(data_path, 'ARC Employment Scenarios.xlsx')
gb_baseline_file = os.path.join(data_path, 'GVA-Employment-Productivity-All LAs.xlsx')

In [ ]:
baseline = pandas.read_excel(
    gb_baseline_file, 
    sheet_name=['GVA', 'Employment', 'Productivity'], 
    header=6, 
    index_col=0
)

In [ ]:
dfs = []
label_lookup = {
    'GVA': 'GVA (GBP2016m)',
    'Employment': 'Employment (000s)',
    'Productivity': 'Productivity (GBP2016 thousands per person in employment)'
}
for label, df in baseline.items():
    df = df.dropna(
    ).reset_index(
    ).rename(
        columns={'index':'lad_nm'}
    ).melt(
        id_vars=['lad_nm'],
        var_name='year',
        value_name=label_lookup[label]
    )
    df.lad_nm = df.lad_nm.apply(lambda nm: nm.strip())
    df = df.set_index(
        ['year', 'lad_nm']
    )
    dfs.append(df)
    
baseline_all = pandas.concat(dfs, axis=1, levels=['year','lad_nm'])
baseline_all

In [ ]:
40 * 380 # years * Great Britain LADs

In [ ]:
variants = pandas.read_excel(
    arc_scenarios_file,
    sheet_name=['baseline', 'scenario 0', 'scenario 1', 'scenario 2'],
    index_col=0
)

In [ ]:
dfs = []
var_names = (
    'KBS Employment', 
    'Non-KBS employment (000s)', 
    'Employment in Total (000s)', 
    'GVA in KBS', 
    'GVA in non-KBS (GBP2011m)', 
    'GVA in Total (GBP2011m)', 
    'Prod in KBS', 
    'Prod in non-KBS (GBP2011m)', 
    'Prod in Total (GBP2011m)'
)
for scenario, s_df in variants.items():
    s_df.index.name = 'lad_nm'
    s_dfs = []
    for i, var in enumerate(var_names):     
        step = 27
        from_row, to_row = i*step + 1, i*step + 25
        df = s_df.iloc[from_row:to_row, 0:37].copy()
        df.columns = df.iloc[0]
        df = df[2:].reset_index(
        ).melt(
            id_vars=['lad_nm'],
            var_name='year',
            value_name=var
        )
        df.lad_nm = df.lad_nm.apply(lambda nm: nm.strip())
        df.year = df.year.astype('int')
        df['scenario'] = scenario
        df = df.set_index(
            ['scenario', 'year', 'lad_nm']
        )
        s_dfs.append(df)
    s_df_all = pandas.concat(s_dfs, axis=1, levels=['scenario', 'year', 'lad_nm'])
    dfs.append(s_df_all)
        
variants_all = pandas.concat(dfs, axis=0)
variants_all

In [ ]:
37*22*4  # years * Arc LADs * scenarios

## Convert from 2011 GBP to 2016 GBP

(CPI in 2016 / CPI in 2011) × 2011 GBP value = 2016 GBP value

CPIH in 2011 (index all items, 2015=100): 93.6

CPIH in 2016 (index all items, 2015=100): 101

Source: https://www.ons.gov.uk/economy/inflationandpriceindices/timeseries/l522/mm23

In [ ]:
cpi16 = 101
cpi11 = 93.6
cpi11/cpi16

In [ ]:
for y in range(2015,2051,5):
    gva11 = variants_all.loc[('baseline', y, 'Cambridge')]['GVA in Total (GBP2011m)']
    gva16 = baseline_all.loc[(y, 'Cambridge')]['GVA (GBP2016m)']  # £2016m
    est = gva16 * (cpi11/cpi16)
    print("%d, %.3f, %.3f, %.3f, %.3f, %.3f" % (y, gva11, gva16, est, abs(gva11 - est), abs(gva11 - est)/gva11))

In [ ]:
baseline_all['GVA (GBP2011m)'] = baseline_all['GVA (GBP2016m)'] * (cpi11 / cpi16)
baseline_all['Productivity (GBP2011 thousands per person in employment)'] = \
    baseline_all['Productivity (GBP2016 thousands per person in employment)'] * (cpi11 / cpi16)

In [ ]:
baseline_all.head()

### Add LAD codes

In [ ]:
lad_nmcds = pandas.read_csv(os.path.join(data_path, 'lad_nmcd_changes.csv'))

In [ ]:
lad_nmcds = lad_nmcds[['lad11nm', 'lad11cd', 'lad16nm', 'lad16cd']]

In [ ]:
baseline_all_lad_nms = set(baseline_all.reset_index().lad_nm.unique())
all_lad_nms = set(lad_nmcds.lad11nm)

In [ ]:
all_lad_nms - baseline_all_lad_nms

In [ ]:
baseline_all_lad_nms - all_lad_nms

In [ ]:
baseline_all = baseline_all.reset_index()
baseline_all.lad_nm = baseline_all.lad_nm.replace({
    'Anglesey': 'Isle of Anglesey',
    'Dumfries & Galloway': 'Dumfries and Galloway',
    'King`s Lynn and West Norfolk': "King's Lynn and West Norfolk",
    'Rhondda, Cynon, Taff': 'Rhondda Cynon Taf'
})

In [ ]:
baseline_wlad = baseline_all.merge(lad_nmcds, left_on='lad_nm', right_on='lad11nm').drop('lad_nm', axis=1)

In [ ]:
len(baseline_wlad.lad11nm.unique()), len(baseline_wlad), len(baseline_all)

In [ ]:
variants_wlad = variants_all.reset_index().merge(lad_nmcds, left_on='lad_nm', right_on='lad11nm').drop('lad_nm', axis=1)

In [ ]:
variants_wlad

In [ ]:
len(variants_wlad.lad11nm.unique()), len(variants_wlad), len(variants_all)

In [ ]:
baseline_wlad[(baseline_wlad.lad11nm != baseline_wlad.lad16nm) | (baseline_wlad.lad11cd != baseline_wlad.lad16cd)].lad16nm.unique()

## Output data

In [ ]:
baseline_wlad.to_csv(os.path.join(output_path, 'gb_baseline.csv'), index=False)

In [ ]:
variants_wlad.to_csv(os.path.join(output_path, 'arc_variants.csv'), index=False)

### Merged, separate file-per-scenario

In [ ]:
base = baseline_wlad[
    ['year', 'Employment (000s)', 'GVA (GBP2011m)', 'lad11nm', 'lad11cd', 'lad16nm', 'lad16cd']
].rename(columns={
    'Employment (000s)': 'employment', 
    'GVA (GBP2011m)': 'gva'
})
base = base[base.year.isin(range(2015, 2051))]
len(base)

### Project Northern rGVA by LAD and sector

Look at projection for Northern Ireland based on 2015 industry composition, UK average growth

In [ ]:
def download(url, filename, force=False):
    if force or not os.path.exists(filename):
        r = requests.get(url, stream=True)
        with open(filename, 'wb') as fd:
            for chunk in r.iter_content(chunk_size=128):
                fd.write(chunk)

In [ ]:
# Regional gross value added (balanced) by local authority in the UK (released December 2017)
# see https://www.ons.gov.uk/economy/grossvalueaddedgva/datasets/regionalgrossvalueaddedbalancedlocalauthoritiesbynuts1region
# Northern Ireland (1998 to 2017)
rgva_ind_url = "https://www.ons.gov.uk/file?uri=/economy/grossvalueaddedgva/datasets/regionalgrossvalueaddedbalancedlocalauthoritiesbynuts1region/uknnorthernireland/regionalgvabbylauknnorthernireland.xlsx"
download(rgva_ind_url, os.path.join(data_path, 'rgva_ni_industry.xlsx'))

In [ ]:
# Nominal and real regional gross value added (balanced) by industry, NUTS1, NUTS2, NUTS3, 1998-2017
rgva_uk_ind_url = "https://www.ons.gov.uk/file?uri=/economy/grossvalueaddedgva/datasets/nominalandrealregionalgrossvalueaddedbalancedbyindustry/current/nominalandrealregionalgvabbyindustry.xlsx"
download(rgva_uk_ind_url, os.path.join(data_path, 'rgva_uk_industry.xlsx'))

In [ ]:
rgva = pandas.read_excel(
    os.path.join(data_path, 'rgva_uk_industry.xlsx'), 
    sheet_name='Table3c', 
    header=1)

In [ ]:
rgva = rgva[:13783]  # skip footnotes

In [ ]:
# pick division-level values (avoid double-count sectors, total)
p = re.compile('^\d')
rgva = rgva[rgva.SIC07.apply(lambda sic: bool(re.match(p, str(sic))))]

In [ ]:
lad_nuts3_lu_url = "http://geoportal1-ons.opendata.arcgis.com/datasets/e1e5de6c5fcc40c78adb03d84a2d299d_0.csv"
download(lad_nuts3_lu_url, os.path.join(data_path, "lad_nuts_lu.csv"))
lad_nuts = pandas.read_csv(os.path.join(data_path, 'lad_nuts_lu.csv'))
lad_nuts = lad_nuts[['LAD16CD', 'LAD16NM', 'NUTS318CD']].sort_values('LAD16CD').drop_duplicates().rename(columns={
    'LAD16CD': 'lad16cd',
    'LAD16NM': 'lad16nm',
    'NUTS318CD': 'nuts318cd'
})
lad_nuts.head(), len(lad_nuts), len(lad_nuts.lad16cd.unique())

In [ ]:
rgva15 = rgva.merge(
    lad_nuts, left_on='Region code', right_on='nuts318cd', how='outer'
).rename(columns={
    'Region name': 'nuts318nm',
    2015: 'gva15_nuts_division_group',
    'SIC07': 'sic07_division_group',
    'SIC07 description': 'sic07_division_group_description'
})[[
    'lad16cd',
    'lad16nm',
    'nuts318cd',
    'nuts318nm',
    'sic07_division_group',
    'sic07_division_group_description',
    'gva15_nuts_division_group'
]]
rgva15.sic07_division_group = rgva15.sic07_division_group.astype(str)
rgva15

In [ ]:
sector_map = pandas.read_csv(os.path.join('data_as_provided','map_sectors.csv'))
sector_map

In [ ]:
rgva15s = rgva15.merge(sector_map, on='sic07_division_group', how='outer').dropna()
rgva15s

### Disaggregate UK 2015 rGVA to LADs and SIC07 divisions

Very coarse, purely proportional split - could be improved e.g. by using BRES employment industry percentage figures for current split to full divisions

In [ ]:
count_for_disagg = rgva15s.groupby(['nuts318cd', 'sic07_division_group']).count()[['lad16cd']]
count_for_disagg.columns = ['nuts_division_group_count']
count_for_disagg = count_for_disagg.reset_index()
rgva15sd = rgva15s.merge(count_for_disagg,on=['nuts318cd', 'sic07_division_group'], how='left')
rgva15sd

In [ ]:
rgva15sd['gva15_lad_division'] = rgva15sd.gva15_nuts_division_group / rgva15sd.nuts_division_group_count
rgva15_lad_division = rgva15sd[[
    'lad16cd', 'itrc_sector', 'sic07_division', 'is_knowledge_based', 'gva15_lad_division'
]]
rgva15_lad_division.head()

In [ ]:
sum_for_disagg = rgva15_lad_division.groupby(['lad16cd']).sum()[['gva15_lad_division']]
sum_for_disagg.columns = ['gva15_lad_total']
rgva15_lad_division_d = rgva15_lad_division.merge(sum_for_disagg.reset_index(), on='lad16cd')
rgva15_lad_division_d['gva15_lad_division_proportion'] = rgva15_lad_division_d.gva15_lad_division / rgva15_lad_division_d.gva15_lad_total
rgva15_lad_division_d.head()

### Project NI regions as proportion of NI total

Where future NI total is projected assuming NI growth equals GB growth:

NI GVA in 2015 * (GB GVA in future year / GB GVA in 2015) = NI GVA in future year

And NI future regional/sectoral GVA follows the same proportional structure as in 2015.

In [ ]:
ni_rgva = rgva15_lad_division_d[rgva15_lad_division_d.lad16cd.str.startswith('N')].copy()
ni_rgva.sort_values(['lad16cd', 'sic07_division']).head()

In [ ]:
ni_rgva_for_proj = ni_rgva[['lad16cd', 'gva15_lad_total']].drop_duplicates()
ni_rgva_for_proj['gva15_lad_ni_proportion'] = ni_rgva_for_proj.gva15_lad_total / ni_rgva_for_proj.gva15_lad_total.sum()
ni_rgva_for_proj

In [ ]:
total_ni_rgva15 = ni_rgva_for_proj.gva15_lad_total.sum()
total_ni_rgva15

In [ ]:
total_uk_rgva15 = rgva15_lad_division_d.gva15_lad_division.sum()
total_uk_rgva15

In [ ]:
total_gb_rgva15 = total_uk_rgva15 - total_ni_rgva15
total_gb_rgva15

In [ ]:
gb_base = base.copy()[['year', 'lad16cd', 'gva', 'employment']]
gb_base.head()

In [ ]:
gb_growth = gb_base.groupby('year').sum()[['gva']].reset_index()
gb_growth.head()

In [ ]:
dfs = []
for year in gb_growth.year:
    df = ni_rgva_for_proj.copy()
    df['year'] = year
    gb_future = float(gb_growth[gb_growth.year == year].gva)
    ni_future = total_ni_rgva15 * (gb_future / total_gb_rgva15)
    df['gva'] = df.gva15_lad_ni_proportion * ni_future
    dfs.append(df)
    
ni_base = pandas.concat(dfs, axis=0)[['year', 'lad16cd', 'gva']]
ni_base.head()

In [ ]:
full_base = pandas.concat(
    [gb_base, ni_base], axis=0, sort=False
).sort_values(
    ['year', 'lad16cd']
).rename(
    columns={'year': 'timestep',}
)
len(full_base), len(full_base.lad16cd.unique()), len(full_base.timestep.unique())

In [ ]:
391 * 36

In [ ]:
full_base.to_csv(os.path.join(output_path, 'arc_gva_employment__baseline.csv'), index=False)

### Stitch scenario projections with baseline UK (GVA by LAD)

In [ ]:
scenario_base = full_base[~full_base.lad16cd.isin(variants_wlad.lad16cd.unique())]
len(scenario_base)

In [ ]:
(391 - 22) * 36

In [ ]:
scenario_base.head()

In [ ]:
vard = {}

for scenario in ('scenario 0', 'scenario 1', 'scenario 2'):
    var = variants_wlad[
        variants_wlad.scenario == scenario
    ][
        ['year', 'Employment in Total (000s)', 'GVA in Total (GBP2011m)', 'lad16cd']
    ].rename(columns={
        'year': 'timestep',
        'Employment in Total (000s)': 'employment',
        'GVA in Total (GBP2011m)': 'gva'
    })
    var = var[var.timestep.isin(range(2015, 2051))].copy()
    vard[scenario] = var

    print(len(var))

In [ ]:
22 * 36

In [ ]:
vard['scenario 0'].head()

In [ ]:
for scenario, key in (('scenario 0', 'unplanned'), ('scenario 1', 'new-cities'), ('scenario 2', 'expansion')):
    stitch = pandas.concat([scenario_base, vard[scenario]], axis=0, sort=False).sort_values(['timestep', 'lad16cd'])
    stitch.to_csv(os.path.join(output_path, 'arc_gva_employment__{}.csv'.format(key)), index=False)
    print(len(stitch))

### Disaggregate projections by SIC07 division, reaggregate to ITRC sector

Assuming constant sectoral shares of GVA, projected LAD sectoral GVA is (projected LAD GVA * (current LAD sectoral GVA / current LAD GVA).

In [ ]:
full_base_disagg = full_base.merge(rgva15_lad_division_d, on='lad16cd', how='outer')
full_base_disagg = disagg.groupby(
    ['timestep', 'lad16cd', 'itrc_sector']
).sum().reset_index()[
    ['timestep', 'lad16cd', 'itrc_sector', 'gva_lad_division']
].dropna().rename(
    columns={'gva_lad_division': 'gva_per_sector'}
)
full_base_disagg.head()

In [ ]:
disagg_d = {}
for scenario, key in (('scenario 0', 'unplanned'), ('scenario 1', 'new-cities'), ('scenario 2', 'expansion')):
    stitch = pandas.concat([scenario_base, vard[scenario]], axis=0, sort=False).sort_values(['timestep', 'lad16cd'])
    disagg = stitch.merge(rgva15_lad_division_d, on='lad16cd', how='outer')
    disagg['gva_lad_division'] = disagg.gva * disagg.gva15_lad_division_proportion
    output = disagg.groupby(
        ['timestep', 'lad16cd', 'itrc_sector']
    ).sum().reset_index()[
        ['timestep', 'lad16cd', 'itrc_sector', 'gva_lad_division']
    ].dropna().rename(
        columns={'gva_lad_division': 'gva_per_sector'}
    )
    disagg_d[key] = output

In [ ]:
len(output), len(output.timestep.unique()), len(output.lad16cd.unique()), len(output.itrc_sector.unique())

In [ ]:
36 * 391 * 46

In [ ]:
energy_demand_sector_subset = [2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 14, 15, 17, 19, 23, 27, 28, 29, 35, 40, 41]
len(energy_demand_sector_subset)

In [ ]:
ed_base_disagg = full_base_disagg[full_base_disagg.itrc_sector.isin(energy_demand_sector_subset)].rename(
    columns={'itrc_sector': 'sectors'}
)
len(ed_base_disagg)

In [ ]:
36 * 391 * 21

In [ ]:
ed_base_disagg.to_csv(
    os.path.join(output_path, 'arc_gva_sector__baseline.csv'), index=False
)

In [ ]:
for key, df in disagg_d.items():
    df.rename(
        columns={'itrc_sector': 'sectors'}
    ).to_csv(
        os.path.join(output_path, 'arc_gva_sector__{}.csv'.format(key)), index=False
    )